In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun 
import numpy as np
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [5]:
data = np.arange(1,50)
rdd = sc.parallelize(data)
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

Find the sum, average, maximum, minimum, and count

In [6]:
#Sum
rdd.sum()

1225

In [9]:
#avg
rdd.mean()

25.0

In [10]:
#Count
rdd.count()

49

In [11]:
#Min
rdd.min()

1

In [12]:
#Max
rdd.max()

49

Count how many numbers are even vs. odd.

In [23]:
rdd_filtered_even = rdd.filter(lambda x: x%2==0).count()
rdd_filtered_odd = rdd.filter(lambda x: x%2!=0).count()
print("Even Count: ", rdd_filtered_even)
print("Odd Count: ", rdd_filtered_odd)

Even Count:  24
Odd Count:  25


You have the following data of people info ('Name', 'Age'), answer the following questions

In [24]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [39]:
# Find the oldest person
oldest_person=rdd_people.map(lambda x: x[1]).max()
oldest_person1 = rdd_people.filter(lambda x: x[1] ==oldest_person).collect()
print(oldest_person1)

[('Khaled', 40)]


Compute the average age

In [41]:
avg_age = rdd_people.map(lambda x: x[1]).mean()
print("Average Age: ", avg_age)

Average Age:  31.666666666666668


Group all the names by their age

In [43]:
grouped_data = rdd_people.map(lambda x: (x[1], x[0])).groupByKey().mapValues(list).collect()
print(grouped_data)

[(30, ['Mona']), (25, ['Nada ', 'Nada ']), (40, ['Khaled']), (35, ['Ahmed', 'Ahmed'])]


Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [48]:
rdd_russia = sc.textFile("/data/russia.txt")
rdd_russia.collect()

['"Russia is the largest country in the world by land area',
 'Moscow is the capital city of Russia',
 'The Russian language is one of the most widely spoken languages in the world',
 'Russia is known for its rich history and culture',
 'The Trans-Siberian Railway is the longest railway line in the world',
 'Russia has a strong tradition in literature, music and ballet',
 'The country is famous for its cold winters and vast landscapes',
 'Russia is a major player in global energy production',
 '"']

Count the total number of lines.

In [51]:
lines_count = rdd_russia.count()
print("Lines Count: ", lines_count)

Lines Count:  8


Count how many lines contain the word "Russia"

In [55]:
Count_of_lines_containing_Russia = rdd_russia.filter(lambda x: "Russia" in x).count()
print("Count of lines containing 'Russia': ", Count_of_lines_containing_Russia)

Count of lines containing 'Russia':  6


Find the most 5 frequent word in the file.

In [57]:
the_most_5_frequent_word = rdd_russia.flatMap(lambda x: x.split(" ")).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).takeOrdered(5, key=lambda x: -x[1])
print("The most 5 frequent word: ", the_most_5_frequent_word)

The most 5 frequent word:  [('is', 7), ('the', 7), ('in', 5), ('Russia', 4), ('world', 3)]


Tokenize words

In [59]:
Tokenize_words = rdd_russia.flatMap(lambda x: x.split(" ")).collect()
print("Tokenize words: ", Tokenize_words)

Tokenize words:  ['"Russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'Moscow', 'is', 'the', 'capital', 'city', 'of', 'Russia', 'The', 'Russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'Russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'The', 'Trans-Siberian', 'Railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'Russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', 'The', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'Russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production"']


Remove stopwords (a, the, is, to, in, of). 

In [61]:
stopwords = ["a", "the", "is", "to", "in"]
filtered_words = rdd_russia.flatMap(lambda x: x.split(" ")).filter(lambda x: x.lower() not in stopwords).collect()
print("Filtered words: ", filtered_words)

Filtered words:  ['"Russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'Moscow', 'capital', 'city', 'of', 'Russia', 'Russian', 'language', 'one', 'of', 'most', 'widely', 'spoken', 'languages', 'world', 'Russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'Trans-Siberian', 'Railway', 'longest', 'railway', 'line', 'world', 'Russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'Russia', 'major', 'player', 'global', 'energy', 'production"']


Count the frequency of each word

In [63]:
frequency_count = rdd_russia.flatMap(lambda x: x.split(" ")).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).collect()
print("Frequency count of each word: ", frequency_count)

Frequency count of each word:  [('"Russia', 1), ('largest', 1), ('country', 2), ('world', 3), ('by', 1), ('land', 1), ('area', 1), ('capital', 1), ('of', 2), ('Russia', 4), ('language', 1), ('most', 1), ('widely', 1), ('known', 1), ('for', 2), ('history', 1), ('and', 3), ('Trans-Siberian', 1), ('Railway', 1), ('line', 1), ('literature,', 1), ('music', 1), ('famous', 1), ('cold', 1), ('winters', 1), ('landscapes', 1), ('player', 1), ('energy', 1), ('production"', 1), ('is', 7), ('the', 7), ('in', 5), ('Moscow', 1), ('city', 1), ('The', 3), ('Russian', 1), ('one', 1), ('spoken', 1), ('languages', 1), ('its', 2), ('rich', 1), ('culture', 1), ('longest', 1), ('railway', 1), ('has', 1), ('a', 2), ('strong', 1), ('tradition', 1), ('ballet', 1), ('vast', 1), ('major', 1), ('global', 1)]


In [80]:
schema = 'id integer, name string, age integer, salary integer' 
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [66]:
Show_schema = df.printSchema()
first_2_rows = df.show(2)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [67]:
name_salary = df.select("name", "salary").show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [ ]:
from pyspark.sql import functions as fun
avg_salary= df.agg(fun.avg("salary").alias("avg_salary"))
avg_salary.show()


+----------+
|avg_salary|
+----------+
|    6000.0|
+----------+



Filter employees older than 28

In [73]:
employees_older_than_28 = df.filter(df.age > 28).show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [77]:
distinct_values_in_the_name_column = df.select("name").distinct().count()
print("Distinct values in the name column: ", distinct_values_in_the_name_column)

Distinct values in the name column:  4


Group by a the name column and find average salary

In [78]:
name_avg_salary = df.groupBy("name").agg(fun.avg("salary").alias("avg_salary")).show()

+------+----------+
|  name|avg_salary|
+------+----------+
|   Ali|    4000.0|
|Mariam|    6750.0|
|  Omar|    6750.0|
|  Sara|    5000.0|
+------+----------+

